In [86]:
import io
import json
import re
import hashlib
from collections import defaultdict
from enum import Enum, auto
from pathlib import Path

import fitz
import numpy as np
from IPython.display import display
from PIL import Image

In [87]:
def extract_pdf_pages_data(pdf_path):
  """
  Extract text and images from all pages of a PDF.

  Returns:
      [
        {
          "text": str,
          "images": [PIL.Image.Image, ...]
        },
        ...
      ]
  """
  pdf_path = Path(pdf_path)
  doc = fitz.open(pdf_path)

  pages_data = []

  for page_index in range(doc.page_count):
    page = doc.load_page(page_index)

    # ---- TEXT ----
    blocks = page.get_text("blocks")
    _text_for_sorting = []

    for b in blocks:
      x0, y0, x1, y1, text, *_ = b
      if text.strip():
        _text_for_sorting.append((y0, x0, text))

    text = "".join(
        t[2] for t in sorted(_text_for_sorting, key=lambda t: t[:-1])
    )

    # ---- IMAGES ----
    _images_for_sorting = []

    for img in page.get_images(full=True):
      xref = img[0]
      img_name = img[7]

      bbox = page.get_image_bbox(img_name)
      y0, x0 = bbox.y0, bbox.x0

      pix = fitz.Pixmap(doc, xref)

      # Ensure RGB (PNG-safe)
      if pix.colorspace is None or pix.colorspace.n != 3:
        pix = fitz.Pixmap(fitz.csRGB, pix)

      pil_img = Image.open(io.BytesIO(pix.tobytes("png")))

      _images_for_sorting.append((y0, x0, pil_img))

      pix = None

    images = [
        t[2] for t in sorted(_images_for_sorting, key=lambda t: t[:-1])
    ]

    pages_data.append({
        "text": text,
        "images": images
    })

  doc.close()
  return pages_data

In [88]:
class Poster:
  def __init__(self, pdf_path: str, page_no: int, code: str, image):
    self.pdf_path = pdf_path
    self.page_no = page_no
    self.code = code.strip()
    self.image = image

    pdf_name = Path(pdf_path).stem
    safe_code = re.sub(r"[^\w\-]", "_", self.code)

    h = hashlib.md5(
        image.tobytes()
    ).hexdigest()[:5]

    self.image_file = f"{pdf_name}--p{page_no}--{safe_code}--{h}.jpg"

  def __repr__(self):
    return f"<Poster page={self.page_no}, code={self.code}>"

In [89]:
class TextExtractMode(Enum):
  LAST_LINE = auto()
  LAST_LINE_AFTER_COLON = auto()
  SECOND_LAST_LINE = auto()

# pdf_registry

In [90]:
pdfs_multi_per_page = {
    "./data/poster-pdfs/multi-per-page/01_Radha_Krishna_Art_Poster.pdf": {
        "category": "Radha Krishna",
    },
    "./data/poster-pdfs/multi-per-page/02_Radha_Krishna_Art_Poster_Vertical.pdf": {
        "category": "Radha Krishna",
    },
    "./data/poster-pdfs/multi-per-page/03_Mixed_Art_Poster_Chinese_Scenery.pdf": {
        "category": "Mixed Art",
    },
    "./data/poster-pdfs/multi-per-page/04_Ganesh_Art_Poster.pdf": {
        "category": "Ganesh",
    },
    "./data/poster-pdfs/multi-per-page/08_Ambedkar_Buddha_Art_Poster.pdf": {
        "category": "Ambedkar Buddha",
    },
    "./data/poster-pdfs/multi-per-page/08_Horse_Art_Poster.pdf": {
        "category": "Horse",
    },
    "./data/poster-pdfs/multi-per-page/09_Ambedkar_Art_Poster.pdf": {
        "category": "Ambedkar",
    },
    "./data/poster-pdfs/multi-per-page/38_Nature_Art_Poster.pdf": {
        "category": "Nature",
    },
}

In [91]:
pdfs_single_per_page = {
    "./data/poster-pdfs/1-per-page/02_Ram_Catalogue_PDF_Brochure.pdf": {
        "text_mode": TextExtractMode.SECOND_LAST_LINE,
        "category": "Ram",
    },
    "./data/poster-pdfs/1-per-page/29_Chinese_Scenery_Modern_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE_AFTER_COLON,
        "category": "Chinese Scenery",
    },
    "./data/poster-pdfs/1-per-page/30_Chinese_Scenery_Modern_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE_AFTER_COLON,
        "category": "Chinese Scenery",
    },
    "./data/poster-pdfs/1-per-page/31_Chinese_Scenery_Modern_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Chinese Scenery",
    },
    "./data/poster-pdfs/1-per-page/40_Mahakal_Art_Poster_2023.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Mahakal",
    },
    "./data/poster-pdfs/1-per-page/45_Tirupati_Balaji_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Tirupati Balaji",
    },
    "./data/poster-pdfs/1-per-page/46_Ram_Darbar_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Ram Darbar",
    },
    "./data/poster-pdfs/1-per-page/47_Shiva_Family_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Shiva Family",
    },
    "./data/poster-pdfs/1-per-page/48_Hanuman_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Hanuman",
    },
    "./data/poster-pdfs/1-per-page/51_Buddha_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Buddha",
    },
    "./data/poster-pdfs/1-per-page/61_Mixed_Gods_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Mixed Gods",
    },
    "./data/poster-pdfs/1-per-page/72_Buddha_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Buddha",
    },
    "./data/poster-pdfs/1-per-page/75_Khatu_Shyam_Art_Poster.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Khatu Shyam",
    },
}

In [92]:
pdfs_incremental = {
    "./data/poster-pdfs/ocr-needed/39_Modern_Art_Poster_2023.pdf": {
        "start_code": 25284,
        "prefix": "",
        "category": "Modern",
    },
    "./data/poster-pdfs/ocr-needed/43_Modern_Art_Poster.pdf": {
        "start_code": 25725,
        "prefix": "",
        "category": "Modern",
    },
    "./data/poster-pdfs/ocr-needed/44_Modern_Art_Poster.pdf": {
        "start_code": 25569,
        "prefix": "",
        "category": "Modern",
    },
    "./data/poster-pdfs/ocr-needed/49_LGS_Art_Poster_New.pdf": {
        "start_code": 1001,
        "prefix": "LGS",
        "category": "Lakshmi Ganesh Saraswati",
    },
}

In [93]:
PDF_REGISTRY = {
    **pdfs_multi_per_page,
    **pdfs_single_per_page,
    **pdfs_incremental,
}

In [94]:
CATEGORY_REGISTRY = {
    "Radha Krishna": (
        "घर में प्रेम, सामंजस्य और भक्ति का संचार करता है। "
        "राधा कृष्ण पोस्टर दिव्य प्रेम और भावनात्मक संतुलन का प्रतीक हैं।"
    ),
    "Chinese Scenery": (
        "आपके स्थान में शांति और प्राकृतिक संतुलन लाता है। "
        "चीनी प्राकृतिक दृश्य कला शांति, समृद्धि और सजगता को बढ़ावा देती है।"
    ),
    "Ganesh": (
        "भगवान गणेश बाधाओं को दूर करते हैं और सफलता आकर्षित करते हैं। "
        "नए आरंभ और सकारात्मक ऊर्जा के लिए आदर्श पोस्टर।"
    ),
    "Ambedkar Buddha": (
        "ज्ञान, समानता और आंतरिक शक्ति की प्रेरक छवि। "
        "प्रेरणा और विचारशील जीवन के लिए उपयुक्त।"
    ),
    "Horse": (
        "गति, शक्ति और प्रगति का प्रतीक। "
        "घोड़े के पोस्टर ऊर्जा बढ़ाते हैं और आगे बढ़ने की प्रेरणा देते हैं।"
    ),
    "Ambedkar": (
        "बुद्धिमत्ता, साहस और सामाजिक न्याय का सम्मान। "
        "प्रेरणा और आत्मसम्मान के लिए अर्थपूर्ण पोस्टर।"
    ),
    "Nature": (
        "शांति और ताजगी से जुड़ने का माध्यम। "
        "प्राकृतिक पोस्टर तनाव कम करते हैं और दृश्य संतुलन लाते हैं।"
    ),
    "Ram": (
        "भगवान राम धर्म, अनुशासन और नैतिक शक्ति के प्रतीक हैं। "
        "शांति और पारिवारिक मूल्यों के लिए आदर्श।"
    ),
    "Mahakal": (
        "महाकाल समय, शक्ति और निर्भयता का प्रतीक हैं। "
        "आत्मविश्वास बढ़ाने वाली सशक्त आध्यात्मिक उपस्थिति।"
    ),
    "Tirupati Balaji": (
        "धन, भक्ति और मनोकामना पूर्ति से जुड़े। "
        "घर और कार्यस्थल के लिए अत्यंत शुभ।"
    ),
    "Ram Darbar": (
        "आदर्श पारिवारिक मूल्यों और नेतृत्व का संपूर्ण चित्र। "
        "एकता, भक्ति और सामंजस्य को प्रोत्साहित करता है।"
    ),
    "Shiva Family": (
        "शक्ति, करुणा और परिवार के संतुलन का प्रतीक। "
        "घर में आध्यात्मिक स्थिरता लाता है।"
    ),
    "Hanuman": (
        "भक्ति, शक्ति और सुरक्षा का प्रतीक। "
        "साहस, एकाग्रता और भय से मुक्ति के लिए उपयुक्त।"
    ),
    "Buddha": (
        "शांति, सजगता और ज्ञानोदय का प्रतीक। "
        "ध्यान कक्ष और शांत वातावरण के लिए उपयुक्त।"
    ),
    "Mixed Gods": (
        "विभिन्न देवी-देवताओं की सामूहिक दिव्य ऊर्जा। "
        "एक ही चित्र में संपूर्ण आध्यात्मिक संतुलन के लिए।"
    ),
    "Khatu Shyam": (
        "आस्था, चमत्कार और अटूट भक्ति के लिए प्रसिद्ध। "
        "भक्तों के लिए आशा और भावनात्मक शक्ति प्रदान करता है।"
    ),
    "Modern": (
        "आधुनिक कला जो शैली और व्यक्तित्व जोड़ती है। "
        "आधुनिक इंटीरियर और रचनात्मक स्थानों के लिए आदर्श।"
    ),
    "Lakshmi Ganesh Saraswati": (
        "धन, विद्या और सफलता का शक्तिशाली संगम। "
        "समृद्धि और शिक्षा के लिए अत्यंत लाभकारी।"
    ),
    "Mixed Art": (
        "जिसका कोई निश्चित विषय नहीं होता। "
        "इसमें गणेश, बुद्ध, घोड़े, प्रकृति या आधुनिक कला कुछ भी हो सकता है।"
    ),
}

# multi_per_page

In [95]:
def extract_multi_per_page(pdf_path, options):
  data = extract_pdf_pages_data(pdf_path)
  result = []

  for page_idx, page in enumerate(data[1:], start=2):
    images = page["images"]
    n = len(images)
    codes = page["text"].splitlines()[:-1][-n:]

    for code, image in zip(codes, images):
      result.append(
          Poster(pdf_path, page_idx, code, image)
      )

  return result

# single_per_page

In [96]:
def extract_single_per_page(pdf_path, options):
  mode = options["text_mode"]

  data = extract_pdf_pages_data(pdf_path)
  result = []

  for page_idx, page in enumerate(data, start=1):
    lines = page["text"].splitlines()
    if not lines:
      continue

    if mode == TextExtractMode.LAST_LINE:
      code = lines[-1]
    elif mode == TextExtractMode.LAST_LINE_AFTER_COLON:
      code = lines[-1].split(":", 1)[1].strip()
    elif mode == TextExtractMode.SECOND_LAST_LINE:
      code = lines[-2]

    image = page["images"][-1]

    result.append(
        Poster(pdf_path, page_idx, code, image)
    )

  return result

# incremental_ocr_needed

In [97]:
def extract_incremental(pdf_path, options):
  start_code = options["start_code"]
  prefix = options.get("prefix", "")

  data = extract_pdf_pages_data(pdf_path)
  result = []

  code = start_code

  for page_idx, page in enumerate(data, start=1):
    image = page["images"][-1]
    full_code = f"{prefix}-{code}" if prefix else str(code)

    result.append(
        Poster(pdf_path, page_idx, full_code, image)
    )

    code += 1

  return result

# generate_all_posters 

In [98]:
def generate_all_posters():
  pipelines = [
      (pdfs_multi_per_page, extract_multi_per_page),
      (pdfs_single_per_page, extract_single_per_page),
      (pdfs_incremental, extract_incremental),
  ]

  all_posters = []

  for registry, extractor in pipelines:
    for pdf_path, options in registry.items():
      print(f"Processing: {pdf_path}")
      posters = extractor(pdf_path, options)
      all_posters.extend(posters)

  print(f"Total posters: {len(all_posters)}")
  return all_posters

In [99]:
all_posters = generate_all_posters()

Processing: ./data/poster-pdfs/multi-per-page/01_Radha_Krishna_Art_Poster.pdf
Processing: ./data/poster-pdfs/multi-per-page/02_Radha_Krishna_Art_Poster_Vertical.pdf
Processing: ./data/poster-pdfs/multi-per-page/03_Mixed_Art_Poster_Chinese_Scenery.pdf
Processing: ./data/poster-pdfs/multi-per-page/04_Ganesh_Art_Poster.pdf
Processing: ./data/poster-pdfs/multi-per-page/08_Ambedkar_Buddha_Art_Poster.pdf
Processing: ./data/poster-pdfs/multi-per-page/08_Horse_Art_Poster.pdf
Processing: ./data/poster-pdfs/multi-per-page/09_Ambedkar_Art_Poster.pdf
Processing: ./data/poster-pdfs/multi-per-page/38_Nature_Art_Poster.pdf
Processing: ./data/poster-pdfs/1-per-page/02_Ram_Catalogue_PDF_Brochure.pdf
Processing: ./data/poster-pdfs/1-per-page/29_Chinese_Scenery_Modern_Art_Poster.pdf
Processing: ./data/poster-pdfs/1-per-page/30_Chinese_Scenery_Modern_Art_Poster.pdf
Processing: ./data/poster-pdfs/1-per-page/31_Chinese_Scenery_Modern_Art_Poster.pdf
Processing: ./data/poster-pdfs/1-per-page/40_Mahakal_Art_Po

# save_poster_images

In [100]:
def save_poster_images(
    all_posters,
    output_dir="frontend/public/poster-images",
):
  output_dir = Path(output_dir)
  output_dir.mkdir(parents=True, exist_ok=True)

  for poster in all_posters:
    path = output_dir / poster.image_file

    if path.exists():
      continue

    poster.image.convert("RGB").save(
        path,
        format="JPEG",
        quality=85,
        subsampling=2,
        optimize=True,
        progressive=True,
    )

In [101]:
poster_to_image = save_poster_images(all_posters)

# generate_pdf_json

In [102]:
def generate_pdf_json(
    all_posters,
    output_dir="frontend/public/pdfs-data",
):
  output_dir = Path(output_dir)
  output_dir.mkdir(parents=True, exist_ok=True)

  grouped = defaultdict(list)

  for poster in all_posters:
    grouped[poster.pdf_path].append({
        "page_no": poster.page_no,
        "code": poster.code,
        "image_file": poster.image_file,
    })

  pdfs_metadata = []

  for pdf_path, posters_data in grouped.items():
    pdf_id = Path(pdf_path).stem
    category = PDF_REGISTRY[pdf_path]["category"]

    pdf_obj = {
        "id": pdf_id,
        "path": pdf_path,
        "category": category,
        "total_posters": len(posters_data),
        "thumbnail": posters_data[0]["image_file"],
        "posters": posters_data,
    }

    # full PDF data (with posters)
    with open(output_dir / f"{pdf_id}.json", "w", encoding="utf-8") as f:
      json.dump(pdf_obj, f, indent=2, ensure_ascii=False)

    # metadata pdf entry (drop posters)
    pdfs_metadata.append({
        k: v for k, v in pdf_obj.items() if k != "posters"
    })

  metadata = {
      "pdfs": pdfs_metadata,
      "categories": [
          {"name": name, "description": desc}
          for name, desc in CATEGORY_REGISTRY.items()
      ],
  }

  # metadata.json
  with open(output_dir / "metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

In [103]:
generate_pdf_json(all_posters)